# http://darden.hatenablog.com/entry/2017/05/26/234845

In [1]:
import array
import random
import json
import numpy as np
import matplotlib.pyplot as plt

from math import sqrt

from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

In [2]:
creator.create('FitnessMin', base.Fitness, weights=(-1.0, -1.0))
creator.create('Individual', array.array, typecode='d', fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox

In [4]:
BOUND_LOW, BOUND_UP = 0.0, 1.0
NDIM = 30

def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]


In [5]:
toolbox.register('attr_float', uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

toolbox.register('evaluate', benchmarks.zdt1)
toolbox.register('mate', tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register('mutate', tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)
toolbox.register('select', tools.selNSGA2)

In [6]:
seed = 64
random.seed(seed)

NGEN = 250
MU = 100
CXPB = 0.9


In [7]:
stats = tools.Statistics(lambda ind:ind.fitness.values)
stats.register('min', np.min, axis=0)
stats.register('max', np.max, axis=0)

In [8]:
logbook = tools.Logbook()
logbook.header = 'gen', 'evals', 'min', 'max'

pop = toolbox.population(n=MU)
pop_ini = pop[:]
pop_ini

[array('d', [0.47615826222934565, 0.6302190543188851, 0.40396096956847294, 0.9200240126763269, 0.01685162040226995, 0.9876105507362524, 0.7041144439033412, 0.8039949881475503, 0.8031057995895188, 0.20136711521936546, 0.8194173185392594, 0.6529689056798653, 0.1474309569583241, 0.8613314138109517, 0.44003630577221464, 0.7212885992572478, 0.08366591038506865, 0.679464779137179, 0.005343265292397037, 0.06382836531522529, 0.10541265149933376, 0.04572002552476451, 0.5888810847998094, 0.21924990817330914, 0.19673746145383764, 0.8149524482305617, 0.7696797658382238, 0.39701291838018926, 0.2137573979486891, 0.7423923323846316]),
 array('d', [0.5496789770341175, 0.35882007710050945, 0.3070191349638173, 0.36420126027137356, 0.4867175409136222, 0.1789634902035938, 0.0016410517036646866, 0.4654349622200482, 0.21349210455755507, 0.05598080636124514, 0.2807600588893786, 0.6727775726121318, 0.8294395863702971, 0.02953604401855603, 0.8063693191035562, 0.30305248405685614, 0.8570748128842592, 0.88690670

In [10]:
invalid_ind = [ind for ind in pop if not ind.fitness.valid]
print(invalid_ind)

[array('d', [0.47615826222934565, 0.6302190543188851, 0.40396096956847294, 0.9200240126763269, 0.01685162040226995, 0.9876105507362524, 0.7041144439033412, 0.8039949881475503, 0.8031057995895188, 0.20136711521936546, 0.8194173185392594, 0.6529689056798653, 0.1474309569583241, 0.8613314138109517, 0.44003630577221464, 0.7212885992572478, 0.08366591038506865, 0.679464779137179, 0.005343265292397037, 0.06382836531522529, 0.10541265149933376, 0.04572002552476451, 0.5888810847998094, 0.21924990817330914, 0.19673746145383764, 0.8149524482305617, 0.7696797658382238, 0.39701291838018926, 0.2137573979486891, 0.7423923323846316]), array('d', [0.5496789770341175, 0.35882007710050945, 0.3070191349638173, 0.36420126027137356, 0.4867175409136222, 0.1789634902035938, 0.0016410517036646866, 0.4654349622200482, 0.21349210455755507, 0.05598080636124514, 0.2807600588893786, 0.6727775726121318, 0.8294395863702971, 0.02953604401855603, 0.8063693191035562, 0.30305248405685614, 0.8570748128842592, 0.886906706

In [11]:
fitness = toolbox.map(toolbox.evaluate, invalid_ind)
print(fitness)

In [13]:
for ind, fit in zip(invalid_ind, fitness):
    ind.fitness.values = fit

pop = toolbox.select(pop, len(pop))
print(pop)

[array('d', [0.007769974775848176, 0.15060535720163015, 0.5265886273479476, 0.7899824983063617, 0.29465948651944485, 0.11884164463949065, 0.3681159159538079, 0.8458231950521432, 0.9288497748278114, 0.11892476987978073, 0.07087492968579756, 0.5732639315610167, 0.27914169190990956, 0.3248944118667405, 0.9496277989797225, 0.9240544477997874, 0.49468553202743926, 0.5092825444925246, 0.6348271197621452, 0.1427336802501531, 0.9002459050732753, 0.5752371180199207, 0.8803835993636965, 0.1797161602060965, 0.23372694084715318, 0.8321241663532488, 0.11802516096067373, 0.6343649792465086, 0.5547883955231075, 0.882530331061678]), array('d', [0.4205827871880261, 0.16193685965703863, 0.31101784008952227, 0.9642170055391386, 0.09900512644083947, 0.08146457710643173, 0.0975276531168473, 0.36904695994942693, 0.9756922571099429, 0.5242340718108146, 0.17076717746586123, 0.262103966196346, 0.4684096838713072, 0.9691841930257938, 0.802576026701756, 0.5130191980464148, 0.3769084803485364, 0.6058741879225162,